In [3]:
from google.cloud import aiplatform
from google.oauth2 import service_account

# 1. Path to your key
KEY_PATH = "/Users/dlangford/Desktop/gcp/gen-lang-client-0085631554-8408cebe1fe7.json"

# 2. Create the credentials object
creds = service_account.Credentials.from_service_account_file(KEY_PATH)

In [9]:
# 1. Initialize Vertex AI
# Replace with your project ID
PROJECT_ID = "gen-lang-client-0085631554" 
REGION = "us-east1"
aiplatform.init(project=PROJECT_ID, location=REGION, credentials=creds, staging_bucket='gs://tpu-jax-bucket')

# 2. Define the Image Path (from your previous 'push' step)
CONTAINER_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/simulations/covid-jax-tpu:latest"

# 3. Create the Custom Job
# This 'packages' your container into a job definition
job = aiplatform.CustomContainerTrainingJob(
    display_name="covid-jax-tpu-simulation",
    container_uri=CONTAINER_URI,
)

# 4. Run the Job on a TPU v5e
# 'ct5lp-hightpu-1t' is the machine type for a TPU v5e (1 chip)
# accelerator_type 'TPU_V5E' tells Vertex AI to attach the hardware
# args passes your config path to your script's argparse
model_job = job.run(
    args=["--config_file", "reference/novel-respiratory-advanced-example-config.json"],
    replica_count=1,
    machine_type="ct5lp-hightpu-1t",
    accelerator_type="TPU_V5_LITEPOD",
    accelerator_count=1,
    sync=False # Setting to False lets your notebook keep running while the job starts
)

print(f"Job submitted! Monitor it here: {model_job.resource_name}")

Training Output directory:
gs://tpu-jax-bucket/aiplatform-custom-training-2026-01-12-15:22:46.748 


AttributeError: 'NoneType' object has no attribute 'resource_name'

In [ ]:
from google.cloud import storage

# Initialize client
client = storage.Client(credentials=creds)
bucket = client.bucket("tpu-jax-bucket")

# List all files in that specific job directory
prefix = "aiplatform-custom-training-2026-01-12-15:22:46.748/"
blobs = bucket.list_blobs(prefix=prefix)

for blob in blobs:
    print(f"Found file: {blob.name}")
    # To download a specific file:
    # blob.download_to_filename(blob.name.split('/')[-1])

NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/tpu-jax-bucket/o?projection=noAcl&prefix=aiplatform-custom-training-2026-01-12-15%3A11%3A33.063%2F&prettyPrint=false: The specified bucket does not exist.